# 1 Basic Vectorization

Vectorization refers to processing multiple compute-intensive steps in parallel. This is done by formulating by computation steps in terms of vectors and using efficient math like dot-product to speed up calculation

In [1]:
from time import perf_counter
import numpy as np

### Comparing speedup for loop-based vs matmul based dot-product for 1D array
- Two random np.arrays a, b are created each of size 10, 100, 1000, 10000, 100000
- Time required to obtain dot product using loop is calculated as mean of 1000 runs to reduce variance
- The same procedure is done for np.matmul operation -> a @ b.T
- Speedup is represented as multiples: Ratio of ```time_taken_for_loop / time_taken_for_matmul```

In [85]:
SIZES_OF_ARRAY = [10, 100, 1000, 10000, 100000]

for size_of_array in SIZES_OF_ARRAY:
    a = np.random.random(size_of_array)
    b = np.random.random(size_of_array)

    avg_loop_time = []
    for _ in range(1000): # Doing same computation for 100 times to reduce runtime variance
        dot_loop_value = 0.0
        t_loop_start = perf_counter()
        for index in range(a.shape[0]):
            dot_loop_value += a[index] * b[index]
        t_loop_end = perf_counter()
        
        t_loop_total = t_loop_end - t_loop_start
        if _ > 3: # Ignoring first 3 loop iterations, to further reduce the noise
            avg_loop_time.append(t_loop_total)
    avg_loop_time = np.mean(avg_loop_time)
    print(f"Size of a, b: {size_of_array}| Dot Product| Value: {dot_loop_value:.2E}, Loop time: {avg_loop_time:.2E}s")

    avg_matmul_time = []
    for _ in range(1000): # Doing same computation for 100 times to reduce runtime variance
        t_matmul_start = perf_counter()
        dot_np_value = a @ b.T
        t_matmul_end = perf_counter()
        
        t_matmul_total = t_matmul_end - t_matmul_start
        if _ > 3: # Ignoring first 3 loop iterations, to further reduce the noise
            avg_matmul_time.append(t_matmul_total)
    avg_matmul_time = np.mean(avg_matmul_time)
    print(f"Size of a, b: {size_of_array}| Dot Product| Value: {dot_np_value:.2E}, Matmul time: {avg_matmul_time:.2E}s")

    speedUp = (t_loop_total / t_matmul_total)
    print(f"SpeedUp of matmul compared to loop for size {size_of_array}: {round(speedUp, 2)} x\n")

Size of a, b: 10| Dot Product| Value: 2.76E+00, Loop time: 2.37E-06s
Size of a, b: 10| Dot Product| Value: 2.76E+00, Matmul time: 1.42E-06s
SpeedUp of matmul compared to loop for size 10: 1.83 x

Size of a, b: 100| Dot Product| Value: 2.59E+01, Loop time: 2.10E-05s
Size of a, b: 100| Dot Product| Value: 2.59E+01, Matmul time: 1.15E-06s
SpeedUp of matmul compared to loop for size 100: 16.83 x

Size of a, b: 1000| Dot Product| Value: 2.38E+02, Loop time: 2.14E-04s
Size of a, b: 1000| Dot Product| Value: 2.38E+02, Matmul time: 1.30E-06s
SpeedUp of matmul compared to loop for size 1000: 158.84 x

Size of a, b: 10000| Dot Product| Value: 2.53E+03, Loop time: 2.19E-03s
Size of a, b: 10000| Dot Product| Value: 2.53E+03, Matmul time: 2.57E-06s
SpeedUp of matmul compared to loop for size 10000: 850.95 x

Size of a, b: 100000| Dot Product| Value: 2.50E+04, Loop time: 2.17E-02s
Size of a, b: 100000| Dot Product| Value: 2.50E+04, Matmul time: 7.51E-05s
SpeedUp of matmul compared to loop for size 1

### Comparing speedup for loop-based vs np.matmul for matrix multiplication of N-D array
- Two random np.arrays a, b are created each of size 10, 100, 1000, 10000, 100000
- Time required to obtain dot product using loop is calculated as mean of 1000 runs to reduce variance
- The same procedure is done for np.matmul operation -> a @ b.T
- Speedup is represented as multiples: Ratio of ```time_taken_for_loop / time_taken_for_matmul```

In [95]:
SIZES_OF_ARRAY = [10, 100, 1000, 10000, 100000]

for size_of_array in SIZES_OF_ARRAY:
    a = np.random.random((size_of_array, size_of_array))
    b = np.random.random((size_of_array, size_of_array))

    avg_loop_time = []
    for _ in range(20): # Doing same computation for 20 times to reduce runtime variance
        result = []
        t_loop_start = perf_counter()
        for i in range(a.shape[0]):
            row = []
            for j in range(b.shape[1]):
                product = 0
                for k in range(a.shape[1]):
                    product += a[i][k] * b[k][j]
                row.append(product)
            result.append(row)
        
        t_loop_end = perf_counter()
        
        t_loop_total = t_loop_end - t_loop_start
        if _ > 3: # Ignoring first 3 loop iterations, to further reduce the noise
            avg_loop_time.append(t_loop_total)
    avg_loop_time = np.mean(avg_loop_time)
    print(f"Size of a, b: {size_of_array}| Loop time: {avg_loop_time:.2E}s")

    avg_matmul_time = []
    for _ in range(20): # Doing same computation for 20 times to reduce runtime variance
        t_matmul_start = perf_counter()
        dot_np_value = a @ b
        t_matmul_end = perf_counter()
        
        t_matmul_total = t_matmul_end - t_matmul_start
        if _ > 3: # Ignoring first 3 loop iterations, to further reduce the noise
            avg_matmul_time.append(t_matmul_total)
    avg_matmul_time = np.mean(avg_matmul_time)
    print(f"Size of a, b: {size_of_array}| np.matmul time: {avg_matmul_time:.2E}s")

    speedUp = (t_loop_total / t_matmul_total)
    print(f"SpeedUp of matmul for N-D array compared to loop for size {size_of_array}: {round(speedUp, 2)} x\n")

Size of a, b: 10| Loop time: 4.24E-04s
Size of a, b: 10| np.matmul time: 1.65E-06s
SpeedUp of matmul for N-D array compared to loop for size 10: 246.94 x

Size of a, b: 100| Loop time: 4.07E-01s
Size of a, b: 100| np.matmul time: 1.04E-04s
SpeedUp of matmul for N-D array compared to loop for size 100: 3999.59 x



KeyboardInterrupt: 